In [ ]:
import os

from gait_pipeline import entropy_dataframe
from construct_dataset import load_data_from_json


filename = './Datasets/left_thigh_1.json'

In [ ]:
full_dataset = load_data_from_json(filename)

In [ ]:
# Calculate persistence diagram entropies, takes ~10 minutes to calculate.

entropies_df = entropy_dataframe(full_dataset)

In [ ]:
import json

filename = './Datasets/Entropies/lt1.json'
with open(filename, 'w+') as fp:
    json.dump(entropies_df, fp, indent=2)

In [ ]:
from construct_dataset import read_data_json
from gait_pipeline import make_entropy_dataframe

entropies = read_data_json('./Datasets/Entropies/lt1.json')
entropy_df = make_entropy_dataframe(entropies)

for key in full_dataset.keys():
    entropy_df[f'sub{key}'] = entropy_df['subject'].apply(lambda x: True if x == key else False)

entropy_df.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix

filename = './Reports/lt1.txt'
if os.path.isfile(filename):
    os.remove(filename)
        
for key in full_dataset.keys():
    X = entropy_df[['H0 Entropy','H1 Entropy', 'H2 Entropy']]
    y = entropy_df[f'sub{key}']

    X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=.33)

    clf = DecisionTreeClassifier(max_depth=3)

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    Title = f'Subject {key} Classification Report'
    ClassRpt = classification_report(y_test, y_pred, zero_division = 0)
    ConMtx = confusion_matrix(y_test,y_pred)

    with open(filename, 'a+') as fp:
        fp.write(Title+'\n')
        fp.write(str(ClassRpt)+'\n')
        fp.write(str(ConMtx)+'\n\n')